In [5]:
!pip install transformers datasets scikit-learn pandas tqdm


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\IKBAR\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
import os
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
import time
from tqdm import tqdm

class SequenceClassificationDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs['input_ids'])

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs['input_ids'][idx],
            'attention_mask': self.inputs['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

class RobertaFineTuning:
    def __init__(self, dataset_path, train_file, validation_file, feature_col, label_col, model_name, batch_size, learning_rate, num_epochs, max_len, optimizer='AdamW', device='cpu'):
        self.dataset_path = dataset_path
        self.train_file = train_file
        self.validation_file = validation_file
        self.feature_col = feature_col
        self.label_col = label_col
        self.model_name = model_name
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.max_len = max_len
        self.optimizer_type = optimizer
        self.device = torch.device(device)

        self.tokenizer = RobertaTokenizer.from_pretrained(self.model_name)

        self.train_df = pd.read_csv(os.path.join(self.dataset_path, self.train_file))
        self.validation_df = pd.read_csv(os.path.join(self.dataset_path, self.validation_file))

        self.num_labels = len(self.train_df[self.label_col].unique())

        self.tokenized_train = self.tokenize_dataset(self.train_df, self.feature_col, self.label_col)
        self.tokenized_validation = self.tokenize_dataset(self.validation_df, self.feature_col, self.label_col)

        self.model_config = RobertaConfig.from_pretrained(self.model_name, num_labels=self.num_labels)
        self.model = RobertaForSequenceClassification.from_pretrained(self.model_name, config=self.model_config)
        self.model.to(self.device)

        if self.optimizer_type == 'Adam':
            self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        elif self.optimizer_type == 'AdamW':
            self.optimizer = AdamW(self.model.parameters(), lr=self.learning_rate)
        else:
            raise ValueError("Unsupported optimizer type. Use 'Adam' or 'AdamW'.")

        self.train_dataloader = self.create_dataloader(self.tokenized_train)
        self.validation_dataloader = self.create_dataloader(self.tokenized_validation, shuffle=False)

    def tokenize_dataset(self, df, feature_col, label_col):
        tokens = self.tokenizer(
            list(df[feature_col]),
            padding=True,
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )
        return tokens, list(df[label_col])

    def create_dataloader(self, tokenized_dataset, shuffle=True):
        dataset = SequenceClassificationDataset(tokenized_dataset[0], tokenized_dataset[1])
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=shuffle)

    def evaluate_model(self, dataloader):
        self.model.eval()
        all_labels, all_predictions = [], []

        with torch.no_grad():
            for batch in dataloader:
                inputs = {key: value.to(self.device) for key, value in batch.items()}
                labels = inputs["labels"]
                outputs = self.model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
                logits = outputs.logits
                _, predicted = torch.max(logits, 1)

                all_labels.extend(labels.cpu().numpy())
                all_predictions.extend(predicted.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_predictions)
        precision = precision_score(all_labels, all_predictions, average='weighted')
        recall = recall_score(all_labels, all_predictions, average='weighted')
        f1 = f1_score(all_labels, all_predictions, average='weighted')

        return accuracy, precision, recall, f1

    def train(self):
        for epoch in range(self.num_epochs):
            self.model.train()
            train_losses = []

            for batch in tqdm(self.train_dataloader, desc=f'Epoch {epoch + 1}/{self.num_epochs}'):
                inputs = {key: value.to(self.device) for key, value in batch.items()}
                outputs = self.model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=inputs["labels"])
                loss = outputs.loss
                train_losses.append(loss.item())

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

            validation_losses = []
            val_acc, val_prec, val_rec, val_f1 = self.evaluate_model(self.validation_dataloader)

            for batch in self.validation_dataloader:
                inputs = {key: value.to(self.device) for key, value in batch.items()}
                outputs = self.model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=inputs["labels"])
                validation_losses.append(outputs.loss.item())

            print(f"Epoch {epoch+1}/{self.num_epochs} - Train Loss: {sum(train_losses)/len(train_losses):.4f} - Val Loss: {sum(validation_losses)/len(validation_losses):.4f} - Val Acc: {val_acc:.4f} - Val F1: {val_f1:.4f}")

    def save_model(self, directory):
        os.makedirs(directory, exist_ok=True)
        self.model.save_pretrained(directory)
        self.tokenizer.save_pretrained(directory)

# === MAIN EKSEKUSI ===
start_time = time.time()
model = 'roberta'
model_name = 'roberta-base'

learning_rate = 2e-5
num_epochs = 3
batch_size = 6
max_len = 512
optimizer = 'AdamW'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

absolute_path = "D:/Berkas/Code/CryptoNew/"
dataset_path = absolute_path + "Datasets/"
train_file = 'train_set.csv'
validation_file = 'validation_set.csv'
feature_col = 'text'
label_col = 'sentiment_numerical'
trained_model = model + '_optimizer_' + optimizer + '_lr_' + str(learning_rate) + '_epochs_' + str(num_epochs) + '_bs_' + str(batch_size) + '_maxlen_' + str(max_len)

classifier = RobertaFineTuning(dataset_path, train_file, validation_file, feature_col, label_col, model_name, batch_size, learning_rate, num_epochs, max_len, optimizer, device)

classifier.train()
classifier.save_model(absolute_path + 'TrainedModels/' + trained_model)

print("Training time: {:.2f} seconds".format(time.time() - start_time))


ImportError: cannot import name 'RobertaForSequenceClassification' from 'transformers' (C:\py_packages\Python311\site-packages\transformers\__init__.py)

In [ ]:
import os
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# === KONFIGURASI ===
absolute_path = "D:/Berkas/Code/CryptoNew/"
test_file = 'test_set.csv'
trained_model_name = 'roberta_optimizer_AdamW_lr_2e-05_epochs_3_bs_6_maxlen_512'  # GANTI nama folder model sesuai modelmu

# Baca data uji
test_df = pd.read_csv(os.path.join(absolute_path, 'Datasets', test_file))

# Load model & tokenizer dari folder hasil fine-tuning
model_path = os.path.join(absolute_path, 'TrainedModels', trained_model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Tokenisasi teks uji
tokenized_test = tokenizer(
    list(test_df['text']),
    padding=True,
    truncation=True,
    return_tensors='pt',
    max_length=512
)

# Pindahkan token ke device (GPU/CPU)
tokenized_test = {k: v.to(device) for k, v in tokenized_test.items()}

# Prediksi
model.eval()
with torch.no_grad():
    outputs = model(**tokenized_test)
    logits = outputs.logits
    _, predicted_labels = torch.max(logits, dim=1)

# Tambahkan kolom prediksi ke DataFrame
output_col = trained_model_name + '_prediction'
test_df[output_col] = predicted_labels.cpu().numpy()

# Simpan ke file
output_file = f'test_set_{trained_model_name}.csv'
test_df.to_csv(os.path.join(absolute_path, 'Datasets', output_file), index=False)

print(f"Prediksi selesai disimpan ke {output_file}")


Prediksi selesai disimpan ke test_set_roberta_optimizer_AdamW_lr_3e-05_epochs_3_bs_6_maxlen_512.csv


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Path ke file hasil prediksi
file_path = "D:/Berkas/Code/CryptoNew/Datasets/test_set_roberta_optimizer_AdamW_lr_2e-05_epochs_3_bs_6_maxlen_512.csv"

# Baca file
df = pd.read_csv(file_path)

# Ambil label asli dan prediksi
y_true = df['sentiment_numerical']
y_pred = df['roberta_optimizer_AdamW_lr_3e-05_epochs_3_bs_6_maxlen_512_prediction']

# Hitung metrik
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

# Tampilkan hasil
print(f"Accuracy :  {accuracy:.4f}")
print(f"Precision:  {precision:.4f}")
print(f"Recall   :  {recall:.4f}")
print(f"F1 Score :  {f1:.4f}")


Accuracy :  0.3330
Precision:  0.1109
Recall   :  0.3330
F1 Score :  0.1664


C:\py_packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
